# Text Extraction

#### Reference Code:
 * https://github.com/SridharPandian/Smart-PDF-Engine/blob/master/Keyword_extractor.py
 * https://github.com/rajeshshanmugam-s/Keyword-Extraction-using-NLTK/blob/master/Textract.ipynb
 * https://stackoverflow.com/questions/17098675/searching-text-in-a-pdf-using-python
 * https://github.com/mstamy2/PyPDF2/issues/254

In [1]:
#!pip install pyPDF2

In [2]:
import os
import glob
import pandas as pd
import numpy as np

import PyPDF2 as pdf
import pandas as pd
import textract

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk

from pathlib import Path
from collections import Counter

import re

In [3]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
#nltk.download('stopwords')

In [5]:
files_path="./data/"
pdf_files= glob.glob(os.path.join(files_path,"*.pdf"))

#### [INFO] PDF with issues while using pyPDF2 (TypeError: ord() expected string of length 1, but int found)

* Short fiber reinforced composites for fused deposition modeling.pdf
* Modeling of Multiple Pore Ceramic Materials Fabricated Via Fused Deposition Process.pdf

In [6]:
read_pdfs = []
for i in pdf_files:
    file = open(i, 'rb')
    pdf_file = pdf.PdfFileReader(file)
    read_pdfs.append(pdf_file)

In [7]:
length = len(read_pdfs)
print(length)

pdf_id = list(range(1, length+1))

117


In [8]:
pdf_files = [x.replace(files_path, "") for x in pdf_files]
pdf_files = [x.replace(".pdf", "") for x in pdf_files]

In [9]:
df = pd.DataFrame( { 'pdf_id': pdf_id , 'pdf_files': pdf_files, 'read_pdfs': read_pdfs } )

print(df.shape)
df.head(5)

(117, 3)


,pdf_id,pdf_files,read_pdfs
0,1,The Effects of Processing Parameters on Defect...,<PyPDF2.pdf.PdfFileReader object at 0x7fa35168...
1,2,3D Printed Glass Surface Finish and Bulk Prope...,<PyPDF2.pdf.PdfFileReader object at 0x7fa35167...
2,3,Numerical study on the effects of hierarchical...,<PyPDF2.pdf.PdfFileReader object at 0x7fa3510a...
3,4,Comparison of Residual Stresses in Inconel 718...,<PyPDF2.pdf.PdfFileReader object at 0x7fa35167...
4,5,Fatigue characterization of Titanium Ti-6Al-4V...,<PyPDF2.pdf.PdfFileReader object at 0x7fa35167...


In [10]:
df.to_csv( 'material_classification.csv' , index = False )

In [11]:
page_count = []
for i in read_pdfs:
    page_count.append(i.getNumPages())

In [12]:
pdf_text = []
for i in read_pdfs:
    count = 0
    text = ""
    while count < i.getNumPages():
        pageObj = i.getPage(count)
        count+=1
        text += pageObj.extractText()
    pdf_text.append(text)
    
print('Done')

Done


In [13]:
def KeywordSearchInPDF(filename, search_words):
    count = 0
    text = ""
    while count < filename.getNumPages():
        pageObj = filename.getPage(count)
        count +=1
        text += pageObj.extractText()
    #if text != "":
    #   text = text
    #else:
    #   text = textract.process(filename, method='tesseract', language='eng')
    tokens = word_tokenize(text)
    punctuations = ['(',')','.','.,','-',';',':','[',']',',','\n','?','!','=','==','<','>','@','#','$','%','^','&','*','.','//','{','}','...','``','+',"''"]
    stop_words = stopwords.words('english')
    keywords = [word for word in tokens if not word in stop_words and  not word in punctuations]
    #print(keywords)
    
    new_dict = {}
    for word in search_words:
        cnt_of_words = keywords.count(word)
        new_dict.update({word: cnt_of_words})
    
    return new_dict 

In [14]:
search_words = ['metal', 'ceramic', 'polymer', 'matrix', 'composite', 'composites',
              'selective', 'laser', 'SLM', 
              'direct', 'sintering', 'DMLS', 
              'fused', 'deposition', 'modeling', 'FDM', 
              'filament', 'fabrication', 'FFF', 
              'extrusion', 'additive', 'manufacturing']

In [15]:
pdf_id = 0
for i in range(0, 10):
    new_dict = {}
    new_dict = KeywordSearchInPDF(read_pdfs[i], search_words)
    print("PDF ID: {}".format(pdf_id))
    print("File name: {}".format(pdf_files[i]))
    print(new_dict)
    print("\n")
    pdf_id += 1

PDF ID: 0
File name: The Effects of Processing Parameters on Defect Regularity in Ti-6Al-4V Parts Fabricated By Selective Laser Melting and Electron Beam Melting
{'metal': 0, 'ceramic': 0, 'polymer': 0, 'matrix': 0, 'composite': 0, 'composites': 0, 'selective': 0, 'laser': 0, 'SLM': 3, 'direct': 0, 'sintering': 0, 'DMLS': 0, 'fused': 0, 'deposition': 0, 'modeling': 0, 'FDM': 0, 'filament': 0, 'fabrication': 0, 'FFF': 0, 'extrusion': 0, 'additive': 0, 'manufacturing': 0}


PDF ID: 1
File name: 3D Printed Glass Surface Finish and Bulk Properties as a Function of the Printing Process
{'metal': 2, 'ceramic': 1, 'polymer': 0, 'matrix': 0, 'composite': 0, 'composites': 0, 'selective': 0, 'laser': 0, 'SLM': 0, 'direct': 1, 'sintering': 0, 'DMLS': 0, 'fused': 2, 'deposition': 0, 'modeling': 0, 'FDM': 0, 'filament': 0, 'fabrication': 0, 'FFF': 0, 'extrusion': 2, 'additive': 1, 'manufacturing': 3}


PDF ID: 2
File name: Numerical study on the effects of hierarchical wavy interface morphology on 